In [115]:
using Revise
using Dates
using Distributions
using Random
using DataFrames
using CSV
using Plots
using StatsBase
using LinearAlgebra
using Statistics
import Pkg
Pkg.activate("..")
Pkg.instantiate()

using SequentialNorta

  Activating project at `c:\Users\ks885\Documents\aa_research\Modeling\spcm_genx_experiment\SPCM\src\scenario_generation\sequential_norta`


In [116]:
scen_generator = scenario_generator_init()

Dict{Any, Any} with 28 entries:
  "solar_actual_avg_cf"         => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.010627…
  "lp_wind"                     => [0.606061 0.707071 … 0.818182 0.99; 0.888889…
  "wind_marginals_by_issue"     => DataFrame[48×103 DataFrame…
  "load_landing_probabilities"  => [0.888889, 0.585859, 0.545455, 0.505051, 0.2…
  "solar_actual_avg"            => 8760×2 DataFrame…
  "lp_load"                     => [0.888889 0.585859 … 0.939394 0.919192; 0.74…
  "solar_well_defined_cols"     => [7, 8, 9, 10, 11, 12, 13, 14, 15, 16  …  35,…
  "solar_model_data"            => 8736×2 DataFrame…
  "load_marginals_by_issue"     => DataFrame[48×103 DataFrame…
  "unique_forecast_times"       => [DateTime("2018-01-01T00:00:00"), DateTime("…
  "M_load"                      => [1.0 0.0 … 0.0 0.0; 0.860519 0.509419 … 0.0 …
  "solar_landing_probabilities" => [0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0.99, 0…
  "forecast_scenario_length"    => 48
  "wind_landing_probabilities"  => [0.606061, 0.70

In [117]:
# Save the objects from scen_generator into individual variables
unique_forecast_times = scen_generator["unique_forecast_times"]
unique_issue_times = scen_generator["unique_issue_times"]
start_date = scen_generator["start_date"]
corr_forecast_issue_times = scen_generator["corr_forecast_issue_times"]
forecast_scenario_length = scen_generator["forecast_scenario_length"]
number_of_scenarios = scen_generator["number_of_scenarios"]
solar_model_data = scen_generator["solar_model_data"]
M_load = scen_generator["M_load"]
M_solar = scen_generator["M_solar"]
M_wind = scen_generator["M_wind"]
load_marginals_by_issue = scen_generator["load_marginals_by_issue"]
solar_marginals_by_issue = scen_generator["solar_marginals_by_issue"]
wind_marginals_by_issue = scen_generator["wind_marginals_by_issue"]
load_landing_probabilities = scen_generator["load_landing_probabilities"]
solar_landing_probabilities = scen_generator["solar_landing_probabilities"]
wind_landing_probabilities = scen_generator["wind_landing_probabilities"]
load_actual_avg = scen_generator["load_actual_avg"]
solar_actual_avg = scen_generator["solar_actual_avg"]
wind_actual_avg = scen_generator["wind_actual_avg"]
solar_well_defined_cols = scen_generator["solar_well_defined_cols"]
load_actual_avg_GW = scen_generator["load_actual_avg_GW"]
solar_actual_avg_cf = scen_generator["solar_actual_avg_cf"]
wind_actual_avg_cf = scen_generator["wind_actual_avg_cf"]
decision_mdl_lkd_length = scen_generator["decision_mdl_lkd_length"];

In [118]:

# Set the date and time for the forecasts
start_index = findfirst(isequal(start_date), unique_forecast_times)

Tstart = start_index;
Tend = decision_mdl_lkd_length - 50; # should be -48 probablys

# Initialize Locked Scenario Path Information
scen_path = Array{Any}(undef, Tend)

Random.seed!(12345);
# intialize normal distribution
normal_dist = Normal(0,1);

In [119]:
M_solar

28×28 LowerTriangular{Float64, Matrix{Float64}}:
  1.0            ⋅           ⋅         …    ⋅            ⋅           ⋅ 
  0.238126      0.971234     ⋅              ⋅            ⋅           ⋅ 
  0.0612079     0.237465    0.969466        ⋅            ⋅           ⋅ 
  0.0563054     0.119579    0.542002        ⋅            ⋅           ⋅ 
 -0.0117155    -0.0186392   0.457456        ⋅            ⋅           ⋅ 
  0.0169431    -0.0454672   0.374329   …    ⋅            ⋅           ⋅ 
 -0.0554145    -0.0888184   0.218231        ⋅            ⋅           ⋅ 
  0.00503775   -0.1034      0.218244        ⋅            ⋅           ⋅ 
  0.0682595    -0.0889985   0.145198        ⋅            ⋅           ⋅ 
 -0.0743783    -0.192338    0.298852        ⋅            ⋅           ⋅ 
  ⋮                                    ⋱   ⋮                        
  0.000276425  -0.154649    0.0680437       ⋅            ⋅           ⋅ 
 -0.0420961    -0.0805923   0.0950802  …    ⋅            ⋅           ⋅ 
 -0.0204562    -0.

In [120]:
363*28/2

5082.0

In [121]:
solar_model_data

Row,time_index,avg_actual
,DateTime,Float64
1,2018-01-01T00:00:00,0.0
2,2018-01-01T01:00:00,0.0
3,2018-01-01T02:00:00,0.0
4,2018-01-01T03:00:00,0.0
5,2018-01-01T04:00:00,0.0
6,2018-01-01T05:00:00,0.0
7,2018-01-01T06:00:00,0.0
8,2018-01-01T07:00:00,6.91694
9,2018-01-01T08:00:00,135.23


In [122]:
solar_positive_indices

4880-element Vector{Int64}:
    8
    9
   10
   11
   12
   13
   14
   15
   16
   17
    ⋮
 8723
 8724
 8725
 8726
 8727
 8728
 8729
 8730
 8731

In [123]:
sunny_decision_hours

UndefVarError: UndefVarError: `sunny_decision_hours` not defined

In [172]:
solar_model_data

Row,time_index,avg_actual
,DateTime,Float64
1,2018-01-01T00:00:00,0.0
2,2018-01-01T01:00:00,0.0
3,2018-01-01T02:00:00,0.0
4,2018-01-01T03:00:00,0.0
5,2018-01-01T04:00:00,0.0
6,2018-01-01T05:00:00,0.0
7,2018-01-01T06:00:00,0.0
8,2018-01-01T07:00:00,6.91694
9,2018-01-01T08:00:00,135.23


In [124]:
# get the indices of when solar_model_data is greater than 0
solar_positive_indices = findall(row -> row[:avg_actual] > 0, eachrow(solar_model_data))

4880-element Vector{Int64}:
    8
    9
   10
   11
   12
   13
   14
   15
   16
   17
    ⋮
 8723
 8724
 8725
 8726
 8727
 8728
 8729
 8730
 8731

In [125]:
# define loop as first 20 elements of sunny_decision_hours
R = solar_positive_indices[1:20];

# initialize scenario path for load, solar and wind
scen_path_load = Array{Matrix{Float64}}(undef, Tend, number_of_scenarios)
scen_path_solar = Array{Matrix{Float64}}(undef, Tend, number_of_scenarios)
scen_path_wind = Array{Matrix{Float64}}(undef, Tend, number_of_scenarios)

8686×20 Matrix{Matrix{Float64}}:
 #undef  #undef  #undef  #undef  #undef  …  #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef     #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef     #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef     #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef     #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef  …  #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef     #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef     #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef     #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef     #undef  #undef  #undef  #undef
   ⋮                                     ⋱                          
 #undef  #undef  #undef  #undef  #undef     #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef     #undef  #undef  #undef  #unde

In [126]:
sunny_decision_hours

UndefVarError: UndefVarError: `sunny_decision_hours` not defined

In [127]:
forecast2model_indices = findall(in(unique_forecast_times), load_actual_avg[!, :time_index])


8736-element Vector{Int64}:
    7
    8
    9
   10
   11
   12
   13
   14
   15
   16
    ⋮
 8734
 8735
 8736
 8737
 8738
 8739
 8740
 8741
 8742

In [128]:
solar_actuals_dec_timeseries = solar_actual_avg[forecast2model_indices, :avg_actual]

8736-element Vector{Float64}:
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   6.916941916666667
 135.23035908333335
 345.78694816666666
   ⋮
  72.38918704166666
  55.24584775
   5.323152916666667
   0.0011250000000000001
   0.0
   0.0
   0.0
   0.0
   0.0

In [129]:
# for r in R[1]
r = R[4]

global decision_date = start_date + Dates.Hour(r - 1)

#=======================================================================
DEFINE DATETIMES AND FORECAST VS ACTUAL VS MODEL TIMES OR HOURS
=======================================================================#

horizon_start_index = findfirst(isequal(decision_date), unique_forecast_times);

current_issue = corr_forecast_issue_times[horizon_start_index, :issue_time];

issue_index = findall(x -> x == current_issue, unique_issue_times)[1];
# find next issue time and compare to start date
# next_issue = unique_issue_times[issue_index + 1];
# define active issues (XXX workshop name) as set of forecasts that are available for the hours to use for lookahead
active_issues = [current_issue];
# get the indices of the forecasts of the active issue times
current_forecast_indices = findall(x -> x in active_issues, corr_forecast_issue_times[!,:issue_time]);

# get the actual forecast times of the current forecast indices
current_forecast_times = corr_forecast_issue_times[current_forecast_indices, :forecast_time];
# get the forecast times that are after the start_data
forecast_times_start_incl = filter(x -> x >= decision_date, current_forecast_times);
# calculate the length of the forecast times after the start date
policy_model_length = length(forecast_times_start_incl);
policy_lookahead_length = policy_model_length - 1; # minus one always for the existing lookahead...

# extract the actuals from the last 48 - model_forecast_length
# actuals_length = 48 - policy_model_length + 1;
policy_actuals_length = forecast_scenario_length - policy_lookahead_length;

lookahead_decision_hours = collect(policy_actuals_length:forecast_scenario_length);

load_current_marginals = load_marginals_by_issue[issue_index];
solar_current_marginals = solar_marginals_by_issue[issue_index];
wind_current_marginals = wind_marginals_by_issue[issue_index];

#=======================================================================
GENERATE SCENARIOS FOR LOAD, SOLAR, AND WIND
=======================================================================#
# Y_load =  generate_norta_scenarios(number_of_scenarios, forecast_scenario_length, decision_date, horizon_start_index, 
#                                     active_issues, corr_forecast_issue_times, normal_dist,  
#                                     M_load, load_current_marginals, load_landing_probabilities,
#                                     sunny_decision_hours, false)

# Y_solar = generate_norta_scenarios(number_of_scenarios, forecast_scenario_length, decision_date, horizon_start_index, 
#                                     active_issues, corr_forecast_issue_times, normal_dist,  
#                                     M_solar, solar_current_marginals, solar_landing_probabilities,
#                                     sunny_decision_hours, true)

# Y_wind = generate_norta_scenarios(number_of_scenarios, forecast_scenario_length, decision_date, horizon_start_index, 
#                                     active_issues, corr_forecast_issue_times, normal_dist,  
#                                     M_wind, wind_current_marginals, wind_landing_probabilities,
#                                     sunny_decision_hours, false)

    

Row,forecast_time,issue_time,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15,p_16,p_17,p_18,p_19,p_20,p_21,p_22,p_23,p_24,p_25,p_26,p_27,p_28,p_29,p_30,p_31,p_32,p_33,p_34,p_35,p_36,p_37,p_38,p_39,p_40,p_41,p_42,p_43,p_44,p_45,p_46,p_47,p_48,p_49,p_50,p_51,p_52,p_53,p_54,p_55,p_56,p_57,p_58,p_59,p_60,p_61,p_62,p_63,p_64,p_65,p_66,p_67,p_68,p_69,p_70,p_71,p_72,p_73,p_74,p_75,p_76,p_77,p_78,p_79,p_80,p_81,p_82,p_83,p_84,p_85,p_86,p_87,p_88,p_89,p_90,p_91,p_92,p_93,p_94,p_95,p_96,p_97,p_98,⋯
,DateTime,DateTime,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,2018-01-01T00:00:00,2017-12-31T12:00:00,4498.39,4504.78,4516.86,4572.79,4666.27,4826.55,4833.27,4908.43,4983.03,5036.68,5073.17,5081.99,5162.45,5313.36,5444.5,5464.98,5480.96,5561.93,5577.43,5747.46,5776.21,5785.94,6008.54,6021.8,6101.73,6103.79,6134.4,6148.76,6191.84,6259.5,6270.44,6280.78,6306.09,6354.07,6390.86,6403.67,6408.49,6434.25,6447.26,6487.89,6591.32,6621.72,6658.19,6696.96,6714.75,6842.24,6881.97,6897.3,6904.32,6953.91,7098.02,7104.45,7128.56,7134.23,7143.28,7158.02,7166.33,7194.31,7273.91,7306.23,7414.59,7430.78,7529.7,7646.74,7679.74,7695.59,7713.04,7811.22,7839.44,7893.65,7974.33,8101.7,8138.01,8166.42,8180.02,8187.36,8213.83,8257.2,8303.84,8343.91,8378.41,8414.76,8426.27,8453.93,8466.78,8472.46,8566.51,8573.84,8593.09,8606.12,8631.82,8663.37,8668.25,8674.34,8879.61,8895.87,8912.22,8990.43,⋯
2,2018-01-01T01:00:00,2017-12-31T12:00:00,4437.08,4442.95,4446.17,4487.63,4756.88,4793.18,4794.78,4836.27,4836.51,4844.16,4879.82,4905.77,4925.04,5010.3,5016.55,5053.17,5067.24,5101.19,5222.99,5279.27,5297.94,5351.71,5378.54,5460.84,5503.84,5506.66,5546.4,5667.55,5701.67,5719.92,5763.83,5795.63,5813.4,5892.43,5939.56,5983.76,5990.77,6006.9,6035.04,6043.89,6073.43,6077.27,6100.61,6117.52,6117.92,6134.41,6148.61,6153.7,6220.07,6253.49,6327.54,6377.47,6397.92,6446.58,6463.91,6503.54,6523.05,6564.83,6689.44,6689.8,6730.35,6730.57,6760.06,6780.57,6842.62,6862.02,6894.51,6894.99,6902.28,6933.18,6984.54,7172.59,7264.91,7265.88,7363.87,7377.31,7388.37,7418.33,7432.59,7436.29,7579.55,7585.19,7632.06,7632.34,7636.73,7900.09,7914.45,7923.18,7954.31,7960.33,7994.93,8012.72,8017.49,8021.83,8071.5,8179.66,8186.33,8267.1,⋯
3,2018-01-01T02:00:00,2017-12-31T12:00:00,3856.06,4056.18,4064.34,4119.03,4140.09,4264.26,4415.73,4445.26,4484.64,4553.4,4565.05,4659.15,4793.43,4819.98,4840.54,4867.77,4892.41,4907.79,4968.18,5015.26,5031.67,5052.19,5104.61,5122.96,5214.51,5215.8,5225.26,5340.12,5355.1,5366.04,5425.86,5464.65,5513.11,5539.41,5596.39,5702.77,5735.54,5762.08,5776.13,5786.64,5791.69,5828.63,5841.4,5843.56,5856.41,5858.81,5900.06,5918.3,5986.14,6000.32,6046.95,6072.82,6083.92,6134.28,6140.66,6159.8,6182.69,6192.68,6204.89,6217.38,6226.42,6227.23,6231.82,6245.11,6261.11,6275.13,6278.66,6288.96,6295.8,6297.04,6298.45,6327.33,6339.53,6353.1,6360.4,6363.08,6411.68,6412.1,6453.82,6509.6,6520.69,6523.0,6535.86,6560.06,6560.67,6645.25,6684.51,6743.4,6759.23,6772.55,6773.5,6784.9,6797.2,6845.02,6852.32,6886.02,6887.34,6889.18,⋯
4,2018-01-01T03:00:00,2017-12-31T12:00:00,3613.21,3678.05,3734.93,3969.08,3978.12,4009.25,4151.98,4161.03,4186.3,4202.7,4218.73,4224.11,4233.48,4257.24,4332.24,4452.27,4459.54,4504.18,4562.26,4595.43,4608.18,4657.02,4675.1,4679.03,

In [130]:
number_of_scen = number_of_scenarios
scenario_len = forecast_scenario_length
date = decision_date
start_ind = horizon_start_index
active_iss = active_issues
corr_forc_iss_times = corr_forecast_issue_times
d_norm = normal_dist
M = M_solar
current_marginals = solar_current_marginals
landing_probabilities = solar_landing_probabilities
sunny_decision_hours = sunny_decision_hours
is_solar = true

UndefVarError: UndefVarError: `sunny_decision_hours` not defined

In [131]:
percentile_column_index = startswith.(names(current_marginals), "p_");
dim_M = size(M, 1);

if dim_M != scenario_len
    kernel_scenario_len = dim_M;
else
    kernel_scenario_len = scenario_len;
end

# get the indices of the forecasts of the active issue times
curr_forec_indices = findall(x -> x in active_iss, corr_forc_iss_times[!,:issue_time]);

# get the actual forecast times of the current forecast indices
curr_forec_times = corr_forc_iss_times[curr_forec_indices, :forecast_time];
# get the forecast times that are after the start_data
forec_times_start_incl = filter(x -> x >= date, curr_forec_times);
# calculate the length of the forecast times after the start date
policy_model_len = length(forec_times_start_incl);
policy_lookahead_len = policy_model_len - 1; # minus one always for the existing lookahead...
policy_actuals_len = scenario_len - policy_lookahead_len;
first_forec_decision_time_hour = curr_forec_indices[1];
# policy_decn_hours = collect(policy_actuals_len:scenario_len);
forecast_decision_time_hours = collect(first_forec_decision_time_hour:first_forec_decision_time_hour+scenario_len-1)

if is_solar
    decision_time_sunny_hours = intersect(sunny_decision_hours, forecast_decision_time_hours);
    policy_time_sunny_hours = decision_time_sunny_hours .- first_forec_decision_time_hour .+ 1;

    # find the actuals sunny hours before and at the policy start
    kernel_actuals_policy_time_hours = policy_time_sunny_hours[policy_time_sunny_hours .<= policy_actuals_len];
    policy_actuals_kernel_len = length(kernel_actuals_policy_time_hours);
    policy_lookahead_kernel_len = kernel_scenario_len - policy_actuals_kernel_len;
else
    policy_lookahead_kernel_len = policy_model_len - 1;
    policy_actuals_kernel_len = scenario_len - policy_lookahead_kernel_len;
end

# initialize the matrix for the normal random variables for the calculation of Z
W = Matrix{Float64}(undef, number_of_scen, kernel_scenario_len);
# initialize the vector for the cumulative distribution function on Z
cdf_Z_48 = zeros(scenario_len,1);

# initialize the matrix for the scenarios
Y = Matrix{Float64}(undef, number_of_scen, scenario_len);

for nscen in 1:number_of_scen
    # nscen = 1
    # initialize
    array_actual_Z = Array{Float64}(undef, kernel_scenario_len)
    array_actual_phi = Array{Float64}(undef, kernel_scenario_len)

    # checks
    max_actual = Array{Float64}(undef, kernel_scenario_len)
    min_actual = Array{Float64}(undef, kernel_scenario_len)
    # loop through number of scenarios to get W realizations from actuals
    for k in 1:kernel_scenario_len
        current_marginals_dist = current_marginals[k, percentile_column_index]
        vec_current_marginals_dist = collect(current_marginals_dist)
        ecdf_actuals = ecdf(vec_current_marginals_dist)

        actual_lp_via_md = ecdf_actuals(current_marginals[k,:avg_actual])
        if actual_lp_via_md  == 1
            actual_lp_via_md  = 0.99
        elseif actual_lp_via_md  == 0
            actual_lp_via_md  = 0.01
        end
        
        # take inverse
        actual_Z_via_md = quantile(d_norm, actual_lp_via_md)

        array_actual_Z[k] = actual_Z_via_md
    end

    W_from_actuals = inv(M) * array_actual_Z

    # set the historical landing probabilities in the first columns of W
    # W[nscen, 1:policy_actuals_len] = historical_lp; # XXX wrong
    W[nscen, 1:policy_actuals_kernel_len] = W_from_actuals[1:policy_actuals_kernel_len];

    # W_upd = rand(d, solar_policy_forecast_length); # policy_forecast_length
    # the rest of the columns are filled with random numbers
    W_upd = rand(d_norm, policy_lookahead_kernel_len); # policy_forecast_length

    # W[nscen, solar_policy_actuals_len+1:end] = W_upd; # policy_actuals_len
    W[nscen, policy_actuals_kernel_len+1:end] = W_upd; # policy_actuals_len

    W_lookahead = W[nscen, :];

    Z = M * W_lookahead;

    cdf_Z_base = cdf.(d_norm, Z);

    # cdf_Z_48[lookahead_sunny_indices] = cdf_Z; # XXX skip
    if is_solar
        # model_sunny_indices = lookahead_sunny_indices[policy_actuals_len+1:end];
        cdf_Z_48[policy_time_sunny_hours] = cdf_Z_base;
    else
        cdf_Z_48 = cdf_Z_base;
    end

    # current_marginals = marginals_by_issue[issue_index];

    for k in 1:scenario_len
        # k = 1
        # marginal_id = within_forecast_sunny_hours[k];
        # current_marginals_dist = current_marginals[marginal_id, 3:101];
        current_marginals_dist = current_marginals[k, percentile_column_index];
        current_actual = current_marginals[k, :avg_actual];
        
        # calculate 
        if k <= policy_actuals_kernel_len # replace policy_actuals_len with actual values
            Y[nscen, k] = current_actual;
        else
            Y[nscen, k] = quantile(current_marginals_dist, cdf_Z_48[k]);
        end
    end
end

model_indices = (scenario_len - policy_model_len + 1): (scenario_len)
policy_model_Y = Y[:, model_indices];

UndefVarError: UndefVarError: `is_solar` not defined

In [132]:
policy_model_Y

UndefVarError: UndefVarError: `policy_model_Y` not defined

In [133]:
solar_actuals_dec_timeseries[R[4]]

372.167962

In [134]:
percentile_column_index = startswith.(names(current_marginals), "p_");
dim_M = size(M, 1);

if dim_M != scenario_len
    kernel_scenario_len = dim_M;
else
    kernel_scenario_len = scenario_len;
end



28

In [135]:
# get the indices of the forecasts of the active issue times
curr_forec_indices = findall(x -> x in active_iss, corr_forc_iss_times[!,:issue_time])


48-element Vector{Int64}:
    1
    2
    3
    4
    5
    6
    7
    8
    9
   10
    ⋮
 8728
 8729
 8730
 8731
 8732
 8733
 8734
 8735
 8736

In [136]:
# get the actual forecast times of the current forecast indices
curr_forec_times = corr_forc_iss_times[curr_forec_indices, :forecast_time]

48-element Vector{DateTime}:
 2018-01-01T00:00:00
 2018-01-01T01:00:00
 2018-01-01T02:00:00
 2018-01-01T03:00:00
 2018-01-01T04:00:00
 2018-01-01T05:00:00
 2018-01-01T06:00:00
 2018-01-01T07:00:00
 2018-01-01T08:00:00
 2018-01-01T09:00:00
 ⋮
 2018-01-02T15:00:00
 2018-01-02T16:00:00
 2018-01-02T17:00:00
 2018-01-02T18:00:00
 2018-01-02T19:00:00
 2018-01-02T20:00:00
 2018-01-02T21:00:00
 2018-01-02T22:00:00
 2018-01-02T23:00:00

In [137]:
# get the forecast times that are after the start_data
forec_times_start_incl = filter(x -> x >= date, curr_forec_times)


38-element Vector{DateTime}:
 2018-01-01T10:00:00
 2018-01-01T11:00:00
 2018-01-01T12:00:00
 2018-01-01T13:00:00
 2018-01-01T14:00:00
 2018-01-01T15:00:00
 2018-01-01T16:00:00
 2018-01-01T17:00:00
 2018-01-01T18:00:00
 2018-01-01T19:00:00
 ⋮
 2018-01-02T15:00:00
 2018-01-02T16:00:00
 2018-01-02T17:00:00
 2018-01-02T18:00:00
 2018-01-02T19:00:00
 2018-01-02T20:00:00
 2018-01-02T21:00:00
 2018-01-02T22:00:00
 2018-01-02T23:00:00

In [138]:
# calculate the length of the forecast times after the start date
policy_model_len = length(forec_times_start_incl)


38

In [139]:
policy_lookahead_len = policy_model_len - 1

37

In [140]:
# minus one always for the existing lookahead...
policy_actuals_len = scenario_len - policy_lookahead_len
# forecast actuals length

11

In [141]:
curr_forec_indices
# raw issue indices

48-element Vector{Int64}:
    1
    2
    3
    4
    5
    6
    7
    8
    9
   10
    ⋮
 8728
 8729
 8730
 8731
 8732
 8733
 8734
 8735
 8736

In [142]:
first_forec_decision_time_hour = curr_forec_indices[1]
# first issue decieion time hour

1

In [143]:
# policy_decn_hours = collect(policy_actuals_len:scenario_len);
forecast_decision_time_hours = collect(first_forec_decision_time_hour:first_forec_decision_time_hour+scenario_len-1)
# indices of current issue

48-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
  ⋮
 40
 41
 42
 43
 44
 45
 46
 47
 48

In [171]:
solar_positive_indices

4880-element Vector{Int64}:
    8
    9
   10
   11
   12
   13
   14
   15
   16
   17
    ⋮
 8723
 8724
 8725
 8726
 8727
 8728
 8729
 8730
 8731

In [170]:
# decision_time_sunny_hours = intersect(solar_positive_indices, forecast_decision_time_hours)
decision_time_sunny_hours = intersect(solar_positive_indices, forecast_decision_time_hours)


24-element Vector{Int64}:
  8
  9
 10
 11
 12
 13
 14
 15
 16
 17
  ⋮
 35
 36
 37
 38
 39
 40
 41
 42
 43

In [145]:
policy_time_sunny_hours = decision_time_sunny_hours .- first_forec_decision_time_hour .+ 1

24-element Vector{Int64}:
  8
  9
 10
 11
 12
 13
 14
 15
 16
 17
  ⋮
 35
 36
 37
 38
 39
 40
 41
 42
 43

In [146]:

# find the actuals sunny hours before and at the policy start
kernel_actuals_policy_time_hours = policy_time_sunny_hours[policy_time_sunny_hours .<= policy_actuals_len]

4-element Vector{Int64}:
  8
  9
 10
 11

In [147]:
policy_actuals_kernel_len = length(kernel_actuals_policy_time_hours)


4

In [148]:
policy_lookahead_kernel_len = kernel_scenario_len - policy_actuals_kernel_len

24

In [149]:
# initialize the matrix for the normal random variables for the calculation of Z
W = Matrix{Float64}(undef, number_of_scen, kernel_scenario_len);
# initialize the vector for the cumulative distribution function on Z
cdf_Z_48 = zeros(scenario_len,1);

# initialize the matrix for the scenarios
Y = Matrix{Float64}(undef, number_of_scen, scenario_len);

In [150]:
nscen = 1

1

In [151]:
# nscen = 1
# initialize
array_actual_Z = Array{Float64}(undef, kernel_scenario_len)
array_actual_phi = Array{Float64}(undef, kernel_scenario_len)

# checks
max_actual = Array{Float64}(undef, kernel_scenario_len)
min_actual = Array{Float64}(undef, kernel_scenario_len)

28-element Vector{Float64}:
  8.672463046105e-312
  4.0714792032809396e198
  4.94e-322
  4.94e-321
  1.235e-321
  1.0e-322
  5.0e-324
  1.6e-322
  8.6721845164e-312
  1.66737e-319
  ⋮
  1.6e-322
 -8.808408122717926e-105
  4.94e-322
  4.94e-321
  1.235e-321
  1.0e-322
  6.83051471218891e110
  1.6e-322
  8.672184516557e-312

In [152]:
k = 1
current_marginals_dist = current_marginals[k, percentile_column_index]


Row,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15,p_16,p_17,p_18,p_19,p_20,p_21,p_22,p_23,p_24,p_25,p_26,p_27,p_28,p_29,p_30,p_31,p_32,p_33,p_34,p_35,p_36,p_37,p_38,p_39,p_40,p_41,p_42,p_43,p_44,p_45,p_46,p_47,p_48,p_49,p_50,p_51,p_52,p_53,p_54,p_55,p_56,p_57,p_58,p_59,p_60,p_61,p_62,p_63,p_64,p_65,p_66,p_67,p_68,p_69,p_70,p_71,p_72,p_73,p_74,p_75,p_76,p_77,p_78,p_79,p_80,p_81,p_82,p_83,p_84,p_85,p_86,p_87,p_88,p_89,p_90,p_91,p_92,p_93,p_94,p_95,p_96,p_97,p_98,p_99
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [153]:
vec_current_marginals_dist = collect(current_marginals_dist)


99-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [154]:
ecdf_actuals = ecdf(vec_current_marginals_dist)


ECDF{Vector{Float64}, Weights{Float64, Float64, Vector{Float64}}}([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Float64[])

In [155]:
actual_lp_via_md = ecdf_actuals(current_marginals[k,:avg_actual])


1.0

In [156]:
if actual_lp_via_md  == 1
    actual_lp_via_md  = 0.99
elseif actual_lp_via_md  == 0
    actual_lp_via_md  = 0.01
end


0.99

In [157]:

# take inverse
actual_Z_via_md = quantile(d_norm, actual_lp_via_md)


2.326347874040846

In [158]:

array_actual_Z[k] = actual_Z_via_md

2.326347874040846

In [159]:
for k in 1:kernel_scenario_len
    current_marginals_dist = current_marginals[k, percentile_column_index]
    vec_current_marginals_dist = collect(current_marginals_dist)
    ecdf_actuals = ecdf(vec_current_marginals_dist)

    actual_lp_via_md = ecdf_actuals(current_marginals[k,:avg_actual])
    if actual_lp_via_md  == 1
        actual_lp_via_md  = 0.99
    elseif actual_lp_via_md  == 0
        actual_lp_via_md  = 0.01
    end
    
    # take inverse
    actual_Z_via_md = quantile(d_norm, actual_lp_via_md)

    array_actual_Z[k] = actual_Z_via_md
end

In [160]:
array_actual_Z[8:22]

15-element Vector{Float64}:
  2.326347874040846
  2.326347874040846
  1.1202053547494895
  0.6665642591979043
  0.6045853465832374
  0.9084578685373853
  1.3996566480498194
  0.5744599931693579
  1.2206403488473494
 -0.24290265588608947
 -1.1202053547494892
  2.326347874040846
  2.326347874040846
  2.326347874040846
  2.326347874040846

In [161]:
W_from_actuals = inv(M) * array_actual_Z

28-element Vector{Float64}:
  2.326347874040846
  1.8248776612177462
  1.8057494888673276
  1.2030395945465042
  1.3148955570374243
  1.1209799808535352
  1.5698489009313792
  0.9648945256766468
  0.8025971028728303
 -0.2795554657954107
  ⋮
  1.5162136797029055
  1.7253193111650404
  1.0346537575644064
  1.1150375745530958
  0.7867769102808666
  2.0758373838957014
  3.1586238914502456
  4.425536291655851
  4.227241297095139

In [162]:
# set the historical landing probabilities in the first columns of W
# W[nscen, 1:policy_actuals_len] = historical_lp; # XXX wrong
W[nscen, 1:policy_actuals_kernel_len] = W_from_actuals[1:policy_actuals_kernel_len]



4-element Vector{Float64}:
 2.326347874040846
 1.8248776612177462
 1.8057494888673276
 1.2030395945465042

In [163]:
# W_upd = rand(d, solar_policy_forecast_length); # policy_forecast_length
# the rest of the columns are filled with random numbers
W_upd = rand(d_norm, policy_lookahead_kernel_len)

24-element Vector{Float64}:
 -1.4680175595540228
 -0.41324194560452254
  0.8393366221167892
  0.4133421496068428
 -0.9869422400282055
 -1.503935416515616
  0.6164765867976383
 -1.4079885254089328
 -0.28810645534662177
  0.39900545762013
  ⋮
 -0.8465877220141987
  2.5436719561116754
  0.07584399073885822
 -0.1460982446438876
 -1.9885871296765367
 -0.9151627050611479
  0.7048468838819091
  0.7180325815727617
 -0.015422794885103095

In [164]:
# W[nscen, solar_policy_actuals_len+1:end] = W_upd; # policy_actuals_len
W[nscen, policy_actuals_kernel_len+1:end] = W_upd

24-element Vector{Float64}:
 -1.4680175595540228
 -0.41324194560452254
  0.8393366221167892
  0.4133421496068428
 -0.9869422400282055
 -1.503935416515616
  0.6164765867976383
 -1.4079885254089328
 -0.28810645534662177
  0.39900545762013
  ⋮
 -0.8465877220141987
  2.5436719561116754
  0.07584399073885822
 -0.1460982446438876
 -1.9885871296765367
 -0.9151627050611479
  0.7048468838819091
  0.7180325815727617
 -0.015422794885103095

In [165]:

W_lookahead = W[nscen, :]

28-element Vector{Float64}:
  2.326347874040846
  1.8248776612177462
  1.8057494888673276
  1.2030395945465042
 -1.4680175595540228
 -0.41324194560452254
  0.8393366221167892
  0.4133421496068428
 -0.9869422400282055
 -1.503935416515616
  ⋮
 -0.8465877220141987
  2.5436719561116754
  0.07584399073885822
 -0.1460982446438876
 -1.9885871296765367
 -0.9151627050611479
  0.7048468838819091
  0.7180325815727617
 -0.015422794885103095

In [166]:

Z = M * W_lookahead

28-element Vector{Float64}:
  2.326347874040846
  2.326347874040846
  2.326347874040846
  2.326347874040846
  0.24538716026551977
  0.1812727567178719
  0.780310814457647
  0.7862437807026101
 -0.43413845902053033
 -1.2975810489591948
  ⋮
 -2.3473491844496404
  0.6493089458630084
  0.08053385460441481
 -0.4435424847283518
 -1.7959663924735283
 -1.9332740517296738
 -1.1541395688241038
  0.6521381755443918
  1.7666600080641492

In [167]:

cdf_Z_base = cdf.(d_norm, Z)

28-element Vector{Float64}:
 0.9900000000000001
 0.9900000000000001
 0.9900000000000001
 0.9900000000000001
 0.5969216651105778
 0.5719232556246292
 0.7823960255836132
 0.7841376592045857
 0.3320939495540039
 0.09721566887479494
 ⋮
 0.009453762953377005
 0.7419306476919846
 0.532093664206159
 0.3286866983213065
 0.03624993106680151
 0.026601223297464234
 0.1242214766241585
 0.7428439806086118
 0.9613574076388969

In [168]:

# model_sunny_indices = lookahead_sunny_indices[policy_actuals_len+1:end];
cdf_Z_48[policy_time_sunny_hours] = cdf_Z_base


DimensionMismatch: DimensionMismatch: tried to assign 28 elements to 24 destinations